In [ ]:
# SDK 2
# Explore datasset
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mltable

# 1. Kết nối Workspace (điền đủ ttin authen để code chạy đc cục bộ)
# ml_client = MLClient.from_config(credential=DefaultAzureCredential())
ml_client = MLClient(
    credential = DefaultAzureCredential(),
    subscription_id = "your azure subscription_id",
    resource_group_name = "your azure resource_group_name",
    workspace_name = "your azure workspace_name",
)

# 2. URI của Data Asset
data_asset = ml_client.data.get("training-data", version="1")
tbl = mltable.load(f'azureml:/{data_asset.id}')
df = tbl.to_pandas_dataframe()
df.head(5)

In [ ]:
# Configure AutoML - classification job
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input, automl

my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:training-data:1")
classification_job = automl.classification(
    compute="ngothilinhau129152",
    experiment_name="auto-ml-class-diabetes",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)
# set limits
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
) 
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)